In [1]:
import sys
import os
# sys.path.append('/home/Nikolay/spark/spark-3.2.3-bin-hadoop3.2/python/lib/py4j-0.10.9.5-src.zip')
# sys.path.append('/home/Nikolay/spark/spark-3.2.3-bin-hadoop3.2/python')
# os.environ["SPARK_HOME"] = '/home/Nikolay/spark/spark-3.2.3-bin-hadoop3.2'
# os.environ["JAVA_HOME"] = '/usr/lib/jvm/java-11-openjdk-11.0.18.0.10-1.fc37.x86_64'
# os.environ["PYSPARK_SUBMIT_ARGS"] ="--master local[2] pyspark-shell"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'ipython ./bin/pyspark'
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ./bin/pyspark'

In [2]:
import warnings
warnings.filterwarnings('ignore')
import time

from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [3]:
spark = SparkSession.builder.master("local").enableHiveSupport().appName("test-1").getOrCreate()
spark

In [4]:
df = spark.read.format("parquet").load('data_in/competition_data_final_pqt/')

In [5]:
# Схема данных
df.printSchema()

root
 |-- region_name: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- cpe_manufacturer_name: string (nullable = true)
 |-- cpe_model_name: string (nullable = true)
 |-- url_host: string (nullable = true)
 |-- cpe_type_cd: string (nullable = true)
 |-- cpe_model_os_type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- date: date (nullable = true)
 |-- part_of_day: string (nullable = true)
 |-- request_cnt: long (nullable = true)
 |-- user_id: long (nullable = true)



В данных имеется: 
    3 числовых столбца
    1 дата
    остальные столбцы со строками

In [7]:
df.show(1, vertical=True)

-RECORD 0-----------------------------------
 region_name           | Краснодарский край 
 city_name             | Краснодар          
 cpe_manufacturer_name | Apple              
 cpe_model_name        | iPhone 7           
 url_host              | ad.adriver.ru      
 cpe_type_cd           | smartphone         
 cpe_model_os_type     | iOS                
 price                 | 20368.0            
 date                  | 2022-06-15         
 part_of_day           | morning            
 request_cnt           | 1                  
 user_id               | 45098              
only showing top 1 row



In [8]:
%%time
# Посмотрим разброс значения для числовых данных
df.select("price", "request_cnt", "user_id").describe().show()

+-------+------------------+------------------+------------------+
|summary|             price|       request_cnt|           user_id|
+-------+------------------+------------------+------------------+
|  count|         316309953|         322899435|         322899435|
|   mean| 33085.37368604712|1.7241966186778865|207098.27418957857|
| stddev|25835.916241525058|1.2138351741492754| 120058.3629955374|
|    min|              90.0|                 1|                 0|
|    max|          195657.0|                16|            415316|
+-------+------------------+------------------+------------------+

CPU times: user 6.79 ms, sys: 5.38 ms, total: 12.2 ms
Wall time: 24.9 s


### Анализ типов устройств

In [11]:
%%time
# Посчитаем количество записей с типом устройства "смартфон" = 322 781 599
df.filter("cpe_type_cd = 'smartphone'").count()

CPU times: user 4.32 ms, sys: 4.57 ms, total: 8.89 ms
Wall time: 15 s


322781599

In [25]:
%%time
# Посчитаем количество записей с различными типа устройств
df.groupBy("cpe_type_cd").count().show(5)

+-----------+---------+
|cpe_type_cd|    count|
+-----------+---------+
|      plain|    36116|
| smartphone|322781599|
|     tablet|    53768|
|    phablet|    27952|
+-----------+---------+

CPU times: user 10.4 ms, sys: 4.33 ms, total: 14.7 ms
Wall time: 27.8 s


Подавляющее количество записей с типом устройства smartphone 322 781 599.

In [32]:
print(f'Количество записей с типом устройства не смартфон: ', 322899435 - 322781599)
print(f'Доля записей с типом устройства смартфон: ', 322781599/322899435)
print(f'Сумма записей с типом устройства plain, tablet, phablet: ', 36116 + 53768 + 27952)

Количество записей с типом устройства не смартфон:  117836
Доля записей с типом устройства смартфон:  0.9996350690424713
Сумма записей с типом устройства plain, tablet, phablet:  117836


In [10]:
%%time
url_host_count = df.select("url_host").distinct().count()
user_id_count = df.select("user_id").distinct().count()
smartphone_user_id_count = df.filter("cpe_type_cd = 'smartphone'").select("user_id").distinct().count()
tablet_user_id_count = df.filter("cpe_type_cd = 'tablet'").select("user_id").distinct().count()
plain_user_id_count = df.filter("cpe_type_cd = 'plain'").select("user_id").distinct().count()
phablet_user_id_count = df.filter("cpe_type_cd = 'phablet'").select("user_id").distinct().count()
print(f"Количество уникальных url_host: {url_host_count}", f"Количество уникальных user_id: {user_id_count}",
f"Доля уникальных user_id использующих смартфон в общем количествек уникальных user_id: {smartphone_user_id_count/user_id_count*100}%",
f"Доля уникальных user_id использующих tablet в общем количествек уникальных user_id: {tablet_user_id_count/user_id_count*100}%",
f"Доля уникальных user_id использующих plain в общем количествек уникальных user_id: {plain_user_id_count/user_id_count*100}%",
f"Доля уникальных user_id использующих phablet в общем количествек уникальных user_id: {phablet_user_id_count/user_id_count*100}%", sep='\n')

Количество уникальных url_host: 199683
Количество уникальных user_id: 415317
Доля уникальных user_id использующих смартфон в общем количествек уникальных user_id: 99.92054262166008%
Доля уникальных user_id использующих tablet в общем количествек уникальных user_id: 0.039728689169959336%
Доля уникальных user_id использующих plain в общем количествек уникальных user_id: 0.02407799343633899%
Доля уникальных user_id использующих phablet в общем количествек уникальных user_id: 0.015891475667983733%
CPU times: user 25.4 ms, sys: 25.9 ms, total: 51.3 ms
Wall time: 2min 48s


In [45]:
%%time
# Посчитаем количество записей с различными ос
df.groupBy("cpe_model_os_type").count().show(5)

+-----------------+---------+
|cpe_model_os_type|    count|
+-----------------+---------+
|              iOS|122764510|
|        Apple iOS|  2275740|
|          Android|197859185|
+-----------------+---------+

CPU times: user 7.93 ms, sys: 5.67 ms, total: 13.6 ms
Wall time: 27.5 s


2 275 740 записей с Apple iOS.
Скорее всего это техническая ошибка, или например, smartphone идет с iOS, а tablet с Apple iOS.

In [48]:
%%time
df.groupBy(["cpe_type_cd", "cpe_model_os_type"]).count().show(10)

+-----------+-----------------+---------+
|cpe_type_cd|cpe_model_os_type|    count|
+-----------+-----------------+---------+
|     tablet|          Android|    51130|
|      plain|          Android|    36116|
|     tablet|              iOS|     2638|
|    phablet|          Android|    27952|
| smartphone|        Apple iOS|  2275740|
| smartphone|          Android|197743987|
| smartphone|              iOS|122761872|
+-----------+-----------------+---------+

CPU times: user 10.3 ms, sys: 6.48 ms, total: 16.8 ms
Wall time: 53.7 s


Как видно из таблицы выше smartphone в основном на iOS, но часть на Apple iOS

In [55]:
%%time
df.filter("cpe_manufacturer_name = 'Apple'").groupBy(["cpe_model_os_type", "cpe_model_name"]).count().orderBy('cpe_model_os_type').show(7)

+-----------------+-----------------+-------+
|cpe_model_os_type|   cpe_model_name|  count|
+-----------------+-----------------+-------+
|        Apple iOS|iPhone 13 Pro Max| 383578|
|        Apple iOS|        iPhone 11| 776193|
|        Apple iOS|        iPhone 13| 638307|
|        Apple iOS|   iPhone 13 mini|  46201|
|        Apple iOS|    iPhone 13 Pro| 431461|
|              iOS|         iPhone 5|  54444|
|              iOS|   iPhone SE 2020|2866247|
+-----------------+-----------------+-------+
only showing top 7 rows

CPU times: user 8.23 ms, sys: 8.92 ms, total: 17.2 ms
Wall time: 34.6 s


Теперь понятно, что это техническая ошибка и для некоторых моделей смартфонов Apple cpe_model_os_type читается как Apple iOS.

<span style="color:red"> **В датасете подготовленном для обучения модели обязательно в столбце cpe_model_os_type Apple iOS исправить на iOS**</span>

### Анализ url_host

In [56]:
%%time
# Посмотрим самые популярные url_host
df.groupBy("url_host").count().orderBy(F.col("count").desc()).show(50)

+--------------------+--------+
|            url_host|   count|
+--------------------+--------+
|googleads.g.doubl...|22013466|
|           yandex.ru|19007657|
|         i.ytimg.com|16901446|
|              vk.com|16695251|
|avatars.mds.yande...|16212095|
|          ad.mail.ru|15309099|
|        yastatic.net|15053937|
|           apple.com| 9743078|
|        ads.adfox.ru| 7771928|
|tpc.googlesyndica...| 7294829|
|  online.sberbank.ru| 7195685|
|       instagram.com| 6936558|
|       zen.yandex.ru| 6367222|
|      mail.yandex.ru| 6123301|
|          google.com| 6014826|
|          icloud.com| 3524545|
|       ad.adriver.ru| 3520373|
|         s0.2mdn.net| 3422837|
|ads.betweendigita...| 3184201|
| sun9-29.userapi.com| 3160532|
| sun9-32.userapi.com| 3159665|
| sun9-17.userapi.com| 3158490|
| sun9-27.userapi.com| 3148620|
|  sun9-5.userapi.com| 3141917|
| sun9-61.userapi.com| 3129243|
|     play.google.com| 2975163|
|          o2.mail.ru| 2640352|
| sun9-77.userapi.com| 2604010|
| sun9-8

In [5]:
%%time
# Посмотрим самые не популярные url_host
df.groupBy("url_host").count().orderBy(F.col("count").asc()).show(50)

+--------------------+-----+
|            url_host|count|
+--------------------+-----+
|     china-bridge.ru|    1|
|           sbelan.ru|    1|
| nash-sovremennik.ru|    1|
|        oldtimer.top|    1|
|         atelecom.ru|    1|
|   vsttorrent.my1.ru|    1|
|varikoz-com.turbo...|    1|
|         vrnchess.ru|    1|
|         us.trud.com|    1|
|      triton-plus.ru|    1|
|       murom.bb24.ru|    1|
|           skipto.ru|    1|
|            sb-78.ru|    1|
|djartik.livejourn...|    1|
|  urbanstreetwear.ru|    1|
|snorbinka-livejou...|    1|
| planeta.strussa.com|    1|
|        blyatukov.ru|    1|
|       chegevara.biz|    1|
|  optk-orsk.narod.ru|    1|
|    nov-n-sh.uobr.ru|    1|
|        tools-ekb.ru|    1|
|khvalynsk.spravke...|    1|
|       povezlovam.ru|    1|
|           sro.press|    1|
|         taxi-178.ru|    1|
|newsru-ca.turbopa...|    1|
| zagorodnaja-life.ru|    1|
|      voilok-wool.ru|    1|
|    rcst.tsu.tula.ru|    1|
| vladimir.4kolesa.co|    1|
|xn----etbbhq1

In [11]:
%%time
# Посчитаем количество url_host с количеством записей = 1
url_host_count_1 = df.groupBy("url_host").count().filter(F.col("count") == 1).count()
print(url_host_count_1)

59247
CPU times: user 12.4 ms, sys: 8.05 ms, total: 20.5 ms
Wall time: 58.9 s


In [12]:
%%time
# Посчитаем долю url_host с количеством записей = 1 в общем количестве уникальных url_host
print( url_host_count_1 / url_host_count * 100, '%')

29.67052778654167 %
CPU times: user 106 µs, sys: 0 ns, total: 106 µs
Wall time: 115 µs


In [13]:
%%time
# Посчитаем долю записей url_host с количеством записей = 1 в общем количестве записей
print( url_host_count_1 / df.count() * 100, '%')

0.018348437184475097 %
CPU times: user 3.48 ms, sys: 2.87 ms, total: 6.35 ms
Wall time: 478 ms


Почти 30% уникальных url_host имеют 1 запись в наборе данных, их доля в общем количестве записей составляет примерно 0,02%.

Можно сделать несколько предположений о том, как обрабатывать такие записи.

<span style="color:red"> **1. Оставить без изменения**</span>

<span style="color:red"> **2. Удалить эти записи, но это повлияет на фичи создаваемые с помощью агрегации**</span>

<span style="color:red"> **3. Переименовать названия указанных url_host в url_host_count_1**</span>

Посмотрим как изменяется количество и доля url_host в зависимости от количества записей от 1 до 100 

In [21]:
#  Общее количество записей
df_count = df.count()

In [52]:
%%time
# Словарь, где ключи - это подсчет количества записей, в которых встречается определенный хост, а значение
#  - это доля хостов, имеющих <= i записей, в общем количестве уникальных хостов
dict_share_url_host = dict()
for i in range(5, 101, 5):
    url_host_count_i = df.groupBy("url_host").count().filter(F.col("count") <= i).count()
    share_url_host = url_host_count_i / url_host_count * 100
    dict_share_url_host[str(i)] = share_url_host

print(dict_share_url_host)

{'5': 61.17045517144675, '10': 72.27104961363762, '15': 77.53038566127312, '20': 80.74297761952694, '25': 83.03410906286463, '30': 84.70976497748933, '35': 86.03186049889075, '40': 87.11006946009425, '45': 88.0099958434118, '50': 88.77220394325005, '55': 89.3976953471252, '60': 89.96559546881807, '65': 90.41681064487213, '70': 90.83697660792356, '75': 91.22859732676292, '80': 91.56713390724298, '85': 91.88864349994742, '90': 92.16508165442225, '95': 92.41898408978231, '100': 92.66687700004506}
CPU times: user 225 ms, sys: 78 ms, total: 303 ms
Wall time: 21min 10s


In [50]:
%%time
# Посчитаем сумму записей url_host с количеством записей <= 100
df.groupBy("url_host").count().filter(F.col("count") <= 100).select(F.sum(F.col("count"))).show()

+----------+
|sum(count)|
+----------+
|   1735069|
+----------+

CPU times: user 16.4 ms, sys: 7.63 ms, total: 24.1 ms
Wall time: 1min 5s


In [51]:
%%time
# Посчитаем долю записей url_host с количеством записей <= 100 в общем количестве записей
print( 1735069 / df.count() * 100, '%')

0.5373403641910987 %
CPU times: user 2.32 ms, sys: 1.32 ms, total: 3.64 ms
Wall time: 305 ms


Почти 93% уникальных url_host имеют <= 100 записей, и их доля в общем числе записей составляет 0,5%.

<span style="color:red"> **Возможно эти 93% уникальных url_host не имееют существенной информации для определения пола пользователя и его возрастной группы**</span>

In [7]:
%%time
df.groupBy("url_host").count().filter(F.col("count") > 1000).count()

CPU times: user 14.4 ms, sys: 6.64 ms, total: 21 ms
Wall time: 1min 1s


3025

In [21]:
%%time
# Создадим список url_host с числом записей больше 1000
df_host_1000 = df.groupBy("url_host").count().filter(F.col("count") > 1000)

CPU times: user 5.17 ms, sys: 2.95 ms, total: 8.12 ms
Wall time: 38.1 ms


In [15]:
df_host_1000.show(10)

+--------------------+------+
|            url_host| count|
+--------------------+------+
| masterh7.adriver.ru| 91193|
|         shopotam.ru|  1555|
|        m.fishki.net| 16083|
|       lightaudio.ru|  8807|
|        altapress.ru|  1893|
|  outlook.office.com|214694|
|  vsrjcrts.tabfil.me|  1560|
|glavbukh-ru.turbo...|  3266|
|       t.newzfeed.ru|  1926|
|             ng72.ru|  1409|
+--------------------+------+
only showing top 10 rows



In [24]:
# Сохраним список url_host с числом записей больше 1000
df_host_1000.repartition(1).write.parquet(path="data_in/url_host_count_1000_pqt", mode="overwrite")

In [25]:
%%time
# Создадим список url_host с числом записей больше 100
df_host_100 = df.groupBy("url_host").count().filter(F.col("count") > 100)

CPU times: user 3.44 ms, sys: 3.54 ms, total: 6.98 ms
Wall time: 29.4 ms


In [18]:
df_host_100.count()

14643

In [26]:
# Сохраним список url_host с числом записей больше 100
df_host_100.repartition(1).write.parquet(path="data_in/url_host_count_100_pqt", mode="overwrite")

In [57]:
%%time
# Посмотрим в какое время дня поступает больше запросов
df.groupBy("part_of_day").count().orderBy(F.col("count").desc()).show()

+-----------+---------+
|part_of_day|    count|
+-----------+---------+
|        day|107328399|
|    evening| 96239286|
|    morning| 85236015|
|      night| 34095735|
+-----------+---------+

CPU times: user 11.2 ms, sys: 5.42 ms, total: 16.6 ms
Wall time: 32.9 s


<span style="color:red"> **Соотношение частоты запросов в разное время суток может коррелировать с возрастными группами, логично предположить, что молодые люди до 30 лет чаще делают запросы ночью, чем люди старшего возраста**</span>

### Мобильность пользователей по регионам и городам

In [58]:
%%time
# Посмотрим из каких регионов чаще всего поступают запросы
df.groupBy("region_name").count().orderBy(F.col("count").desc()).show(10)

+--------------------+--------+
|         region_name|   count|
+--------------------+--------+
|              Москва|45759326|
|  Краснодарский край|26744015|
|  Московская область|20637759|
|     Санкт-Петербург|20120151|
|Республика Башкор...|16014980|
|Республика Татарстан|13225533|
|Свердловская область|11985133|
|  Ростовская область| 9171236|
|Новосибирская обл...| 9089971|
|   Самарская область| 8962704|
+--------------------+--------+
only showing top 10 rows

CPU times: user 6.99 ms, sys: 12.1 ms, total: 19.1 ms
Wall time: 46.5 s


In [ ]:
df.createOrReplaceTempView("mts")

In [76]:
%%time
# Посчитаем пользователей, от которых поступали запросы из одного региона
spark.sql("SELECT count(user_id) from"
          "(SELECT user_id, count(region_name) as region_count from"
          "(SELECT user_id, region_name from mts group by user_id, region_name) as t1 "
          "group by user_id) as t2 where region_count = 1").show(1)

+--------------+
|count(user_id)|
+--------------+
|        268762|
+--------------+

CPU times: user 11.3 ms, sys: 6.72 ms, total: 18 ms
Wall time: 58.9 s


In [78]:
print(f"Доля пользователей, от которых поступали запросы только из одного региона: {268762 / 415317}")
print(f"Доля пользователей, от которых поступали запросы из разных регионов: {146555 / 415317}")

Доля пользователей, от которых поступали запросы только из одного региона: 0.6471249671937339
Доля пользователей, от которых поступали запросы из разных регионов: 0.35287503280626603


In [69]:
spark.sql("SELECT user_id, region_name from mts where user_id = 374036").show(50)

+-------+-------------------+
|user_id|        region_name|
+-------+-------------------+
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Краснодарский край|
| 374036| Краснодарский край|
| 374036| Московская область|
| 374036| Краснодарский край|
| 374036| Краснодарский край|
| 374036| Краснодарский край|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Краснодарский край|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Краснодарский край|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Краснодарский край|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Краснодарский край|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036| Московская область|
| 374036|В

<span style="color:red"> **Поступление запросов из разных регионов может коррелировать с возрастными группами**</span>

In [81]:
%%time
# Посчитаем пользователей, от которых поступали запросы из одного города
spark.sql("SELECT count(user_id) from"
          "(SELECT user_id, count(city_name) as city_count from"
          "(SELECT user_id, city_name from mts group by user_id, city_name) as t1 "
          "group by user_id) as t2 where city_count >= 2").show(1)

+--------------+
|count(user_id)|
+--------------+
|        228209|
+--------------+

CPU times: user 8.65 ms, sys: 8.14 ms, total: 16.8 ms
Wall time: 54.6 s


In [83]:
print(f"Доля пользователей, от которых поступали запросы только из одного города: {187108 / 415317}")
print(f"Доля пользователей, от которых поступали запросы из разных городов: {228209 / 415317}")

Доля пользователей, от которых поступали запросы только из одного города: 0.4505185195886516
Доля пользователей, от которых поступали запросы из разных городов: 0.5494814804113485


<span style="color:red"> **Мобильность пользователей между городами выше чем, мобильность между регионами**</span>

### Изменения цен устройства

In [100]:
%%time
# Посмотрим меняется ли цена устройства в запросах
spark.sql("SELECT count(user_id) from"
          "(SELECT user_id, count(price) as price_count from"
          "(SELECT user_id, price from mts group by user_id, price) as t1 "
          "group by user_id) as t2 where price_count >= 2").show(1)

+--------------+
|count(user_id)|
+--------------+
|          9466|
+--------------+

CPU times: user 4.6 ms, sys: 7.29 ms, total: 11.9 ms
Wall time: 26.4 s


У 395148 пользователей цена не меняется. У 10703 пользователей цена равна null (пример user_id 308767). У 9466 пользователей цена изменяется (пример user_id 303728)

In [102]:
print(f"Контрольная сумма: ", 395148 + 9466 + 10703)

Контрольная сумма:  415317


<span style="color:red"> **Надо подробнее расмотреть пользователей с ценой устройства null, возможно это техническая ошибка, или их цена действительно равна 0. 
Для пользователей с изменяющейся ценой устройства, в качестве цены надо брать медиану.**</span>

### Разрыв в датах

In [11]:
# посчитаем сколько дней у нас в выборке и сравним с количеством дней между мининальной и максимальной датой
spark.sql("SELECT min(date), max(date), count(distinct date), datediff(max(date), min(date))"
          "FROM mts").show()

+----------+----------+--------------------+------------------------------+
| min(date)| max(date)|count(DISTINCT date)|datediff(max(date), min(date))|
+----------+----------+--------------------+------------------------------+
|2021-06-16|2022-11-01|                 396|                           503|
+----------+----------+--------------------+------------------------------+



In [29]:
# найдем самый большой разрыв в датах
spark.sql("with subt as (select distinct date from mts order by date asc)" 
          "select date, pre_date, dif_date from (select date," 
          "lag(date) over (order by date) as pre_date,"
          "date - lag(date) over (order by date) as dif_date from subt) as t "
          "where int(dif_date) >= 2").show(5)

+----------+----------+------------------+
|      date|  pre_date|          dif_date|
+----------+----------+------------------+
|2022-10-06|2022-06-19|INTERVAL '109' DAY|
+----------+----------+------------------+

